In [1]:
pip install transformers datasets torch


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 8.2 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [39]:
from datasets import load_dataset

dataset = load_dataset("squad")


In [40]:
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 87599
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 10570
    })
})

In [31]:
reduced_train_dataset = dataset["train"].select(range(100))
reduced_validation_dataset = dataset["train"].select(range(40))




In [38]:
dataset["train"] = reduced_train_dataset
dataset["validation"] = reduced_validation_dataset
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 37
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 15
    })
})

In [41]:
def filter_one_word_answers(example):
    return all(len(answer.split()) == 1 for answer in example["answers"]["text"])

# Apply the filter to the train and validation datasets
reduced_train_dataset = dataset["train"].filter(filter_one_word_answers)
reduced_validation_dataset = dataset["validation"].filter(filter_one_word_answers)

Filter:   0%|          | 0/87599 [00:00<?, ? examples/s]

Filter:   0%|          | 0/10570 [00:00<?, ? examples/s]

In [42]:
dataset["train"] = reduced_train_dataset
dataset["validation"] = reduced_validation_dataset
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 30267
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 2584
    })
})

In [43]:
dataset["train"][3]

{'id': '5733bed24776f41900661188',
 'title': 'University_of_Notre_Dame',
 'context': 'The university is the major seat of the Congregation of Holy Cross (albeit not its official headquarters, which are in Rome). Its main seminary, Moreau Seminary, is located on the campus across St. Joseph lake from the Main Building. Old College, the oldest building on campus and located near the shore of St. Mary lake, houses undergraduate seminarians. Retired priests and brothers reside in Fatima House (a former retreat center), Holy Cross House, as well as Columba Hall near the Grotto. The university through the Moreau Seminary has ties to theologian Frederick Buechner. While not Catholic, Buechner has praised writers from Notre Dame and Moreau Seminary created a Buechner Prize for Preaching.',
 'question': 'Where is the headquarters of the Congregation of the Holy Cross?',
 'answers': {'text': ['Rome'], 'answer_start': [119]}}

In [4]:
from transformers import T5Tokenizer, T5ForConditionalGeneration, Trainer, TrainingArguments


tokenizer = T5Tokenizer.from_pretrained("t5-small")
model = T5ForConditionalGeneration.from_pretrained("t5-small")

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [44]:
def preprocess_function(examples):
    # Inputs: Combine the passage and answer for the model to generate the question
    inputs = [
        f" Context: {context} Answer: {answer['text'][0]}"
        for context, answer in zip(examples["context"], examples["answers"]) ]

    targets = examples['question']

    # Tokenize inputs and outputs
    model_inputs = tokenizer(inputs, max_length=512, truncation=True, padding="max_length")
    labels = tokenizer(targets, max_length=128, truncation=True, padding="max_length")
    model_inputs["labels"] = labels["input_ids"]

    return model_inputs


tokenized_dataset = dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/30267 [00:00<?, ? examples/s]

Map:   0%|          | 0/2584 [00:00<?, ? examples/s]

In [45]:
tokenized_dataset['train']

Dataset({
    features: ['id', 'title', 'context', 'question', 'answers', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 30267
})

In [ ]:
import os
os.environ["WANDB_DISABLED"] = "false"

In [46]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
)

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
)

# Fine-tune the model
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Epoch,Training Loss,Validation Loss
1,0.247500,0.211099
2,0.233800,0.203629
3,0.214900,0.202037


TrainOutput(global_step=11352, training_loss=0.278860696043407, metrics={'train_runtime': 4293.8703, 'train_samples_per_second': 21.147, 'train_steps_per_second': 2.644, 'total_flos': 1.2289170915459072e+16, 'train_loss': 0.278860696043407, 'epoch': 3.0})

In [52]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

# Load the fine-tuned T5 model and tokenizer
model = T5ForConditionalGeneration.from_pretrained("./results/checkpoint-1000")
tokenizer = T5Tokenizer.from_pretrained("t5-small")

# Prepare input for question generation
input_text = (
    "context: The Lobund Institute grew out of pioneering research in germ-free-life "
    "which began in 1928. This area of research originated in a question posed by Pasteur as to whether "
    "animal life was possible without bacteria. Though others had taken up this idea, their research was short-lived "
    "and inconclusive. Lobund was the first research organization to answer definitively, that such life is possible "
    "and that it can be prolonged through generations. But the objective was not merely to answer Pasteur's question "
    "but also to produce the germ-free animal as a new tool for biological and medical research. answer: 1928"
)

# Tokenize the input
input_ids = tokenizer(input_text, return_tensors="pt").input_ids

# Generate the question
output_ids = model.generate(
    input_ids,
    max_length=128,  # Adjust as needed
    num_beams=4,     # Beam search for better results
    early_stopping=True
)

# Decode the generated tokens to get the predicted question
predicted_question = tokenizer.decode(output_ids[0], skip_special_tokens=True)

# Print the predicted question
print(f"Predicted Question: {predicted_question}")


Predicted Question: What year did The Lobund Institute begin research in germ-free-life?


In [55]:
!zip -r /content/mcqgen2.zip ./results/checkpoint-11000

  adding: results/checkpoint-11000/ (stored 0%)
  adding: results/checkpoint-11000/generation_config.json (deflated 29%)
  adding: results/checkpoint-11000/training_args.bin (deflated 51%)
  adding: results/checkpoint-11000/trainer_state.json (deflated 82%)
  adding: results/checkpoint-11000/rng_state.pth (deflated 25%)
  adding: results/checkpoint-11000/config.json (deflated 62%)
  adding: results/checkpoint-11000/scheduler.pt (deflated 56%)
  adding: results/checkpoint-11000/optimizer.pt (deflated 7%)
  adding: results/checkpoint-11000/model.safetensors (deflated 9%)
